<a href="https://colab.research.google.com/github/hululuzhu/chinese-ai-writing-share/blob/main/Inference_T5_Finetune_Chinese_Couplet_Poem_V2_02_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference for models trained from [T5 chinese couplet colab](https://github.com/hululuzhu/chinese-ai-writing-share/blob/main/Mengzi_T5_Finetune_Chinese_Couplet_V1.ipynb) and [T5 chinese Poem colab](https://github.com/hululuzhu/chinese-ai-writing-share/blob/main/WIP_Mengzi_T5_Finetune_Chinese_Poem_Writing_V1.ipynb)
- Download my saved models at [couplet model link](https://drive.google.com/drive/folders/1bQb_nrHHLkDYj09P2rrX7PSvHD8h3cTx?usp=sharing) and [poem model link](https://drive.google.com/drive/folders/1ZymaSbOcwlslD5tuUIk_9__C2dUJK_UY?usp=sharing)

## Load package and previously trained models

In [ ]:
# Quite install simple T5 package
!pip install -q simplet5 &> /dev/null
!pip install -q chinese-converter > /tmp/na
import chinese_converter  # 繁体到简体需要

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p my_t5/couplet
!mkdir -p my_t5/poem
# 3 epochs, 6 hours P100 16G
!cp /content/drive/MyDrive/ML/Models/t5-couplet/simplet5-epoch-2-train-loss-3.126/* my_t5/couplet
# 4 epochs, 8 hours P100 16G
!cp /content/drive/MyDrive/ML/Models/t5-poem/simplet5-epoch-3-train-loss-3.597/* my_t5/poem

In [ ]:
import torch
from simplet5 import SimpleT5
from transformers import T5Tokenizer, T5ForConditionalGeneration

class MengziSimpleT5(SimpleT5):
  def __init__(self) -> None:
    super().__init__()
    self.device = torch.device("cuda")

  def load_my_model(self, local_path, use_gpu: bool = True):
    self.tokenizer = T5Tokenizer.from_pretrained("Langboat/mengzi-t5-base")
    self.model = T5ForConditionalGeneration.from_pretrained(local_path)

Global seed set to 42


In [ ]:
couplet_model = MengziSimpleT5()
couplet_model.load_my_model(local_path='my_t5/couplet')
couplet_model.model = couplet_model.model.to('cuda')

COUPLET_PROMPOT = '对联：'
MAX_SEQ_LEN = 32
MAX_OUT_TOKENS = MAX_SEQ_LEN

def couplet(in_str, model=couplet_model, is_input_traditional_chinese=False):
  model.model = model.model.to('cuda')
  in_request = f"{COUPLET_PROMPOT}{in_str[:MAX_SEQ_LEN]}"
  if is_input_traditional_chinese:
    # model only knows s chinese
    in_request = chinese_converter.to_simplified(in_request)
  # Note default sampling is turned off for consistent result
  out = model.predict(in_request,
                      max_length=MAX_OUT_TOKENS)[0].replace(",", "，")
  if is_input_traditional_chinese:
    out = chinese_converter.to_traditional(out)
  print(f"上： {in_str}\n下： {out}")

In [ ]:
AUTHOR_PROMPT = "模仿："
TITLE_PROMPT = "作诗："
EOS_TOKEN = '</s>'

poem_model = MengziSimpleT5()
poem_model.load_my_model(local_path='my_t5/poem')
poem_model.model = poem_model.model.to('cuda')
MAX_AUTHOR_CHAR = 4
MAX_TITLE_CHAR = 12
MIN_CONTENT_CHAR = 10
MAX_CONTENT_CHAR = 64

def poem(title_str, opt_author=None, model=poem_model,
         is_input_traditional_chinese=False):
  model.model = model.model.to('cuda')
  if opt_author:
    in_request = TITLE_PROMPT + title_str[:MAX_TITLE_CHAR] + EOS_TOKEN + AUTHOR_PROMPT + opt_author[:MAX_AUTHOR_CHAR]
  else:
    in_request = TITLE_PROMPT + title_str[:MAX_TITLE_CHAR]
  if is_input_traditional_chinese:
    in_request = chinese_converter.to_simplified(in_request)
  out = model.predict(in_request,
                      max_length=MAX_CONTENT_CHAR)[0].replace(",", "，")
  if is_input_traditional_chinese:
    out = chinese_converter.to_traditional(out)
    print(f"標題： {in_request.replace('</s>', ' ')}\n詩歌： {out}")
  else:
    print(f"标题： {in_request.replace('</s>', ' ')}\n诗歌： {out}")

## Inference now
- Note we turned off sampling to see determistic results for comparison

In [ ]:
# epoch 3 after 6 hours, looks good enough
for pre in ['欢天喜地度佳节', '不待鸣钟已汗颜，重来试手竟何艰',
            '当年欲跃龙门去，今日真披马革还', '载歌在谷',
            '北国风光，千里冰封，万里雪飘','寂寞寒窗空守寡',
            '烟锁池塘柳', '五科五状元，金木水火土',
            '望江楼，望江流，望江楼上望江流，江楼千古，江流千古']:
  couplet(pre)

# Support Traditional Chinese
for pre in ['載歌在谷', '飛龍在天', '都說臺北風光好']:
  couplet(pre, is_input_traditional_chinese=True)

上： 欢天喜地度佳节
下： 笑语欢歌迎新春
上： 不待鸣钟已汗颜，重来试手竟何艰
下： 何堪击鼓频催泪?一别伤心更枉然
上： 当年欲跃龙门去，今日真披马革还
下： 此日当登虎榜来，他年又见龙图新
上： 载歌在谷
下： 对酒当歌
上： 北国风光，千里冰封，万里雪飘
下： 南疆气象，五湖浪涌，三江潮来
上： 寂寞寒窗空守寡
下： 逍遥野渡醉吟诗
上： 烟锁池塘柳
下： 云封岭上松
上： 五科五状元，金木水火土
下： 三才三进士，诗书礼乐诗
上： 望江楼，望江流，望江楼上望江流，江楼千古，江流千古
下： 听雨阁，听雨落，听雨阁中听雨落，雨阁万重，雨落万重
上： 載歌在谷
下： 對酒當歌
上： 飛龍在天
下： 臥虎於淵
上： 都說臺北風光好
下： 不曉臺灣景色新


In [ ]:
for title in ['秋思', "百花", '佳人有约']:
  # Empty author means general style
  for author in ['', "杜甫", "李白", "李清照", "苏轼"]:
    poem(title, author)
  print()

for title in ['春節', "中秋"]:
  # Empty author means general style
  for author in ['', "杜甫", "李白", "李清照", "蘇軾"]:
    poem(title, author, is_input_traditional_chinese=True)
  print()

标题： 作诗：秋思
诗歌： 秋思不可奈，况复值新晴。露叶红犹湿，风枝翠欲倾。客愁随日薄，归夢逐云轻。独倚阑干久，西风吹雁声。
标题： 作诗：秋思 模仿：杜甫
诗歌： 西风动高树，落叶满空庭。白露侵肌冷，青灯照眼青。客愁随暮角，归夢逐残星。独坐还成感，秋声不可听。
标题： 作诗：秋思 模仿：李白
诗歌： 秋色满空山，秋风动客衣。浮云不到处，明月自来归。
标题： 作诗：秋思 模仿：李清照
诗歌： 秋思不可奈，况复在天涯。客路逢寒食，家书报早炊。风霜侵鬓发，天地入诗脾。欲寄南飞雁，归期未有期。
标题： 作诗：秋思 模仿：苏轼
诗歌： 西风吹雨过江城，独倚阑干思不胜。黄叶满庭秋意动，碧梧当户夜寒生。故园夢断人千里，新雁书来雁一行。莫怪衰翁无业，一樽聊复慰平生。

标题： 作诗：百花
诗歌： 百花开尽绿阴成，红紫妖红照眼明。谁道东风无意思，一枝春色爲谁荣。
标题： 作诗：百花 模仿：杜甫
诗歌： 百花开尽绿阴成，独有江梅照眼明。莫道春光无别意，只应留得一枝横。
标题： 作诗：百花 模仿：李白
诗歌： 百花如锦树，春色满芳洲。日暖花争发，风轻絮乱流。香飘金谷露，艳拂玉山楼。谁道无情物，年年爲客愁。
标题： 作诗：百花 模仿：李清照
诗歌： 百花如锦水如蓝，春到园林处处堪。谁道东风不相识，一枝开尽绿阴南。
标题： 作诗：百花 模仿：苏轼
诗歌： 百花开尽绿阴成，谁道春风不世情。若使此花无俗韵，世间那得有芳名。

标题： 作诗：佳人有约
诗歌： 佳人约我共登台，笑指花前酒半杯。莫道春光无分到，且看红日上楼来。
标题： 作诗：佳人有约 模仿：杜甫
诗歌： 佳人有约到江干，共约寻春入肺肝。红杏绿桃相映发，白苹红蓼不胜寒。花前醉舞春风裏，月下狂歌夜漏残。莫怪相逢不相识，只应清夢在长安。
标题： 作诗：佳人有约 模仿：李白
诗歌： 佳人有约在瑶台，花落花开不待开。莫道春风无分到，且看明月照楼台。
标题： 作诗：佳人有约 模仿：李清照
诗歌： 佳人约我共登台，花下相携醉不回。莫道春归无觅处，桃花依旧笑人来。
标题： 作诗：佳人有约 模仿：苏轼
诗歌： 佳人约我共清欢，笑指花前醉玉盘。莫道春归无觅处，且看红日上栏干。

標題： 作诗：春节
詩歌： 去年今日到江干，家在青山綠水間。老去心情渾似舊，春來情緒只如閒。
標題： 作诗：春节 模仿：杜甫
詩歌： 江上春歸早，山中客到稀。亂花隨處發，細草向